In [ ]:
%pip install xgboost

In [18]:
import pandas as pd
import numpy as np
import ast
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error




# MODEL - Price - Based Filtering

In [19]:
purchased_games = pd.read_csv("./clean_datasets/purchased_games.csv")  # (playerid, library)
prices = pd.read_csv("./clean_datasets/prices.csv")  # (gameid, date_acquired, price_usd)

In [ ]:
# Tạo DataFrame mở rộng từ danh sách game của mỗi người chơi
# Chỉ cần chạy 1 lần - lưu lại rồi đọc thôithôi
import os

if (not os.path.exists("./clean_datasets/player_data.csv")):
    
    player_data = []
    for _, row in purchased_games.iterrows():
        playerid = row["playerid"]
        
        games = ast.literal_eval(row["library"])  
        
        prices_list = [prices.loc[prices["gameid"] == game, "price_usd"].values[0] 
                    for game in games if game in prices["gameid"].values]

        if len(prices_list) > 0:
            player_data.append({
                "playerid": playerid,
                "mean_price": np.mean(prices_list),
                "median_price": np.median(prices_list),
                "min_price": np.min(prices_list),
                "max_price": np.max(prices_list),
                "last_5_avg": np.mean(prices_list[-5:]) if len(prices_list) >= 5 else np.mean(prices_list),
                "target_price": np.mean(prices_list[-5:])  # Nhãn dự đoán
            })

    player_data


    df1 = pd.DataFrame(player_data)
    # Write 1 lần thôi - nhớ comment lại lần sau chỉ cần đọc file
    df1.to_csv("./data_model/player_data_prices.csv", index=False)
# Hoặc đọc file 

In [20]:
# Đọc file thay vì chạy lại
df= pd.read_csv("./data_model/player_data_prices.csv")

In [21]:
# Chuẩn bị dữ liệu huấn luyện
X = df.drop(columns=["playerid", "target_price"])
y = df["target_price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình XGBoost
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Đánh giá mô hình
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

# Dự đoán giá tối ưu cho người chơi mới
def suggest_game_for_player(playerid):
    # Lấy danh sách game của người chơi từ player_data
    player_info = df[df["playerid"] == playerid]
    
    if player_info.empty:
        return "Không tìm thấy dữ liệu người chơi này."
    
    # Trích xuất thông tin về giá
    mean_price = player_info["mean_price"].values[0]
    median_price = player_info["median_price"].values[0]
    min_price = player_info["min_price"].values[0]
    max_price = player_info["max_price"].values[0]
    last_5_avg = player_info["last_5_avg"].values[0]

    # Dự đoán mức giá phù hợp
    feature_vector = np.array([[mean_price, median_price, min_price, max_price, last_5_avg]])
    predicted_price = model.predict(feature_vector)[0]

    # Tìm các game có giá gần nhất với mức giá này
    suggested_games = prices.iloc[(prices["price_usd"] - predicted_price).abs().argsort()[:10]]  # Lấy 10 game gần nhất
    
    return suggested_games



RMSE: 0.24174656623292673


In [22]:
# DDự đoán cho một người chơi mới với danh sách gameid đã mua
test_player = 76561198060698936
print(suggest_game_for_player(test_player))

        gameid date_acquired  price_usd
49635  1582650    2025-02-24  22.695552
16231   667970    2025-02-24  22.693167
67504  2116800    2025-02-24  22.710504
28225   991780    2025-02-24  22.712488
1361    200510    2025-02-24  22.679996
43721  1417880    2025-02-24  22.674023
28452   998990    2025-02-24  22.723771
66925  2099880    2025-02-24  22.723771
18471   724250    2025-02-24  22.723771
61261  1911610    2025-02-24  22.667224


# MODEL 2 - Content-Based Filtering

In [23]:
import pandas as pd
import re
import ast 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
from sklearn.decomposition import TruncatedSVD


In [25]:
# Chạy file popular game trước để lấy 10 % game phổ biến

purchased_games = pd.read_csv("./clean_datasets/purchased_games.csv")  # playerid, library
top_10_percent_game = pd.read_csv("./data_model/top_10_percent_games.csv")  # gameid, title, genres, description
achievements = pd.read_csv("./clean_datasets/achievements.csv")  # achievementid, gameid, title, description
history = pd.read_csv("./clean_datasets/history.csv")  # playerid, achievementid, date_acquired

In [26]:
def preprocess_text(text):
    """ Hàm tiền xử lý văn bản: chuyển thành chữ thường và loại bỏ dấu câu """
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Loại bỏ khoảng trắng thừa
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ dấu câu
    return text

top_10_percent_game['processed_title'] = top_10_percent_game['title'].apply(preprocess_text)
top_10_percent_game['processed_genres'] = top_10_percent_game['genres'].apply(preprocess_text)
top_10_percent_game['processed_developers'] = top_10_percent_game['developers'].apply(preprocess_text)
top_10_percent_game['processed_publishers'] = top_10_percent_game['publishers'].apply(preprocess_text)
top_10_percent_game['processed_languages'] = top_10_percent_game['supported_languages'].apply(preprocess_text)

# Tạo một cột tổng hợp các thông tin để vector hóa
top_10_percent_game['combined_features'] = top_10_percent_game['processed_title'] + ' ' + \
                              top_10_percent_game['processed_genres'] + ' ' + \
                              top_10_percent_game['processed_developers'] + ' ' + \
                              top_10_percent_game['processed_publishers'] + ' ' + \
                              top_10_percent_game['processed_languages']
top_10_percent_game

,gameid,title,developers,publishers,genres,supported_languages,release_date,purchase_count,processed_title,processed_genres,processed_developers,processed_publishers,processed_languages,combined_features
0,730,Counter-Strike 2,['Valve'],['Valve'],"['Action', 'Free To Play']","['Czech', 'Danish', 'Dutch', 'English', 'Finni...",2012-08-21,43967.0,counterstrike 2,action free to play,valve,valve,czech danish dutch english finnish french germ...,counterstrike 2 action free to play valve valv...
1,578080,PUBG: BATTLEGROUNDS,['PUBG Corporation'],"['KRAFTON, Inc.']","['Action', 'Adventure', 'Massively Multiplayer...","['English', 'Korean', 'Simplified Chinese', 'F...",2017-12-21,28356.0,pubg battlegrounds,action adventure massively multiplayer free to...,pubg corporation,krafton inc,english korean simplified chinese french germa...,pubg battlegrounds action adventure massively ...
2,550,Left 4 Dead 2,['Valve'],['Valve'],['Action'],"['Danish', 'Dutch', 'English', 'Finnish', 'Fre...",2009-11-16,24727.0,left 4 dead 2,action,valve,valve,danish dutch english finnish french german ita...,left 4 dead 2 action valve valve danish dutch ...
3,218620,PAYDAY 2,['OVERKILL - a Starbreeze Studio.'],['Starbreeze Publishing AB'],"['Action', 'RPG']","['English', 'German', 'French', 'Italian', 'Sp...",2013-08-13,23785.0,payday 2,action rpg,overkill a starbreeze studio,starbreeze publishing ab,english german french italian spanish spain d...,payday 2 action rpg overkill a starbreeze stu...
4,304930,Unturned,['Smartly Dressed Games'],['Smartly Dressed Games'],"['Action', 'Adventure', 'Casual', 'Indie', 'Fr...",['English'],2017-07-07,21620.0,unturned,action adventure casual indie free to play,smartly dressed games,smartly dressed games,english,unturned action adventure casual indie free to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9819,440550,Atomik: RunGunJumpGun,['ThirtyThree'],['Good Shepherd Entertainment'],"['Action', 'Indie']","['English', 'French', 'Italian', 'German', 'Sp...",2016-08-31,211.0,atomik rungunjumpgun,action indie,thirtythree,good shepherd entertainment,english french italian german spanish spain j...,atomik rungunjumpgun action indie thirtythree ...
9820,501950,Rogue Islands,['Big Fat Alien'],Unknown,"['Action', 'Adventure', 'Indie', 'RPG', 'Strat...",['English'],2017-09-12,211.0,rogue islands,action adventure indie rpg strategy,big fat alien,unknown,english,rogue islands action adventure indie rpg strat...
9821,462530,8i - Make VR Human,['8i'],['8i'],['Adventure'],['English'],2016-04-25,211.0,8i make vr human,adventure,8i,8i,english,8i make vr human adventure 8i 8i english
9822,1329540,CROSSBOW: Bloodnight,['Hyperstrange'],['Hyperstrange'],['Action'],"['English', 'Polish']",2020-09-24,211.0,crossbow bloodnight,action,hyperstrange,hyperstrange,english polish,crossbow bloodnight action hyperstrange hypers...


In [ ]:
import os
if (not os.path.exists('./data_model/cosine_similarity.csv')):

    # Chạy 1 lần rồi sau đó đọc file thôi
    # Tạo vector TF-IDF cho các đặc trưng tổng hợp của game
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    games_tfidf = tfidf_vectorizer.fit_transform(top_10_percent_game['combined_features'])

    # Tính toán độ tương đồng cosine giữa các game
    cosine_sim = cosine_similarity(games_tfidf)

    # Lưu kết quả cosine similarity vào CSV để tái sử dụng
    cosine_sim_df = pd.DataFrame(cosine_sim, index=top_10_percent_game['gameid'], columns=top_10_percent_game['gameid'])
    cosine_sim_df.to_csv('./data_model/cosine_similarity.csv', index=True)


In [ ]:
cosine_sim_df = pd.read_csv('./data_model/cosine_similarity.csv')
cosine_sim_df

gameid,730,578080,550,218620,304930,1172470,230410,271590,407530,4000,...,827610,680930,444590,1227280,378110,440550,501950,462530,1329540,708920
gameid,,,,,,,,,,,,,,,,,,,,,
730,1.000000,0.231541,0.834606,0.153968,0.030021,0.256917,0.211991,0.187554,0.226253,0.601473,...,0.182583,0.006698,0.006879,0.138216,0.001633,0.158932,0.009044,0.001282,0.017750,0.017089
578080,0.231541,1.000000,0.206550,0.081982,0.032037,0.218618,0.190349,0.133938,0.155101,0.205277,...,0.170563,0.006163,0.011975,0.122090,0.006828,0.166990,0.015743,0.005361,0.016332,0.015724
550,0.834606,0.206550,1.000000,0.155028,0.005416,0.222727,0.182293,0.188846,0.227812,0.605617,...,0.244550,0.006744,0.006927,0.139168,0.001644,0.160027,0.009106,0.001291,0.017872,0.017207
218620,0.153968,0.081982,0.155028,1.000000,0.005579,0.169468,0.107794,0.143688,0.124372,0.106617,...,0.115138,0.006947,0.019063,0.099886,0.001694,0.092161,0.025062,0.001330,0.018412,0.052335
304930,0.030021,0.032037,0.005416,0.005579,1.000000,0.043333,0.032374,0.026435,0.021070,0.010904,...,0.009690,0.024658,0.014287,0.022199,0.006372,0.009487,0.018783,0.005003,0.006257,0.012136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440550,0.158932,0.166990,0.160027,0.092161,0.009487,0.201673,0.156285,0.141409,0.112391,0.158661,...,0.208609,0.011451,0.012133,0.147817,0.001886,1.000000,0.015951,0.001481,0.008418,0.025377
501950,0.009044,0.015743,0.009106,0.025062,0.018783,0.021294,0.026055,0.018055,0.038548,0.007354,...,0.016293,0.006856,0.040624,0.042136,0.025242,0.015951,1.000000,0.008412,0.010520,0.020405
462530,0.001282,0.005361,0.001291,0.001330,0.005003,0.007251,0.001382,0.006148,0.004402,0.001339,...,0.001513,0.001249,0.006398,0.001493,0.006036,0.001481,0.008412,1.000000,0.001491,0.001894


In [29]:
def preprocess_purchased_games(purchased_games):
    def safe_eval(val):
        if isinstance(val, str):  
            try:
                return ast.literal_eval(val)
            except (ValueError, SyntaxError):
                return []  # Trả về danh sách rỗng nếu lỗi
        elif isinstance(val, list):
            return val  
        return []  # Trả về danh sách rỗng nếu không phải list

    purchased_games['library'] = purchased_games['library'].apply(safe_eval)
    return purchased_games


In [30]:
def recommend_games_for_player(playerid, purchased_games, cosine_sim_df, games_df, top_n=10):
    """Gợi ý game dựa trên lịch sử mua game của người chơi."""
    
    # Chuyển playerid và gameid về kiểu string để tránh lỗi
    purchased_games['playerid'] = purchased_games['playerid'].astype(str)
    playerid = str(playerid)

    # Lọc danh sách game mà người chơi đã mua
    player_games = purchased_games[purchased_games['playerid'] == playerid]

    if player_games.empty:
        print(f"⚠️ Không tìm thấy người chơi {playerid} trong danh sách purchased_games.")
        return games_df.head(top_n)[['gameid', 'title', 'genres']]  # Trả về game phổ biến
    
    # Lấy danh sách game đã mua
    purchased_game_ids = player_games['library'].values[0]
    
    # Nếu danh sách rỗng, trả về game phổ biến
    if not purchased_game_ids:
        print(f"⚠️ Người chơi {playerid} chưa mua game nào.")
        return games_df.head(top_n)[['gameid', 'title', 'genres']]
    
    # Tính điểm tương đồng
    similar_scores = pd.Series(dtype=float)
    for game_id in purchased_game_ids:
        if game_id in cosine_sim_df.index:
            similar_scores = pd.concat([similar_scores, cosine_sim_df[game_id]])
        

    # Tổng hợp điểm số, sắp xếp giảm dần
    similar_scores = similar_scores.groupby(similar_scores.index).sum().sort_values(ascending=False)
    
    # Loại bỏ game đã mua, chọn 10 game đầu tiên
    recommended_game_ids = similar_scores[~similar_scores.index.isin(purchased_game_ids)].head(top_n).index
    
    # Nếu không đủ game để gợi ý (vì không tìm thấy trong cosine_sim_df), bổ sung thêm game phổ biến
    if len(recommended_game_ids) < top_n:
        print(f"⚠️ Không đủ game gợi ý. Bổ sung thêm game phổ biến.")
        recommended_game_ids = recommended_game_ids.append(games_df[~games_df['gameid'].isin(purchased_game_ids)].head(top_n - len(recommended_game_ids))['gameid'])
    
    return games_df[games_df['gameid'].isin(recommended_game_ids)][['gameid', 'title', 'genres']]



In [31]:
purchased_games = preprocess_purchased_games(purchased_games)  # Chuyển đổi cột 'library'

In [32]:
playerid = 76561198060698936  
recommended_games = recommend_games_for_player(playerid, purchased_games, cosine_sim_df, top_10_percent_game)
print(recommended_games)

       gameid                title  \
659    527230         For The King   
2095  1272080             PAYDAY 3   
2221  1062830                 Embr   
2334   467950     Survive in Space   
2695  1220150            Blue Fire   
3500  1502190           From Space   
5104   934700        Dead Island 2   
5243  1677280  Company of Heroes 3   
5309  1676840      For The King II   
6219   705120    Death Coming/死神来了   

                                                 genres  
659           ['Adventure', 'Indie', 'RPG', 'Strategy']  
2095                     ['Action', 'Adventure', 'RPG']  
2221  ['Action', 'Casual', 'Indie', 'Simulation', 'S...  
2334  ['Action', 'Adventure', 'Casual', 'Indie', 'RPG']  
2695                   ['Action', 'Adventure', 'Indie']  
3500                                ['Action', 'Indie']  
5104                            ['Action', 'Adventure']  
5243                             ['Action', 'Strategy']  
5309          ['Adventure', 'Indie', 'RPG', 'Strategy']  
6

# MODEL 3 - Collaborative Filtering

In [35]:
%pip install pyspark
%pip install findspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
!python -m ensurepip --upgrade

!python.exe -m pip install --upgrade pip

!python -m pip install --upgrade setuptools

%pip install setuptools --upgrade --force-reinstall


!python -m distutils.core --help

Looking in links: c:\Users\ACER\AppData\Local\Temp\tmpv1mdgwho
  Using cached setuptools-78.1.0-py3-none-any.whl.metadata (6.6 kB)
Using cached setuptools-78.1.0-py3-none-any.whl (1.3 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 78.1.0
    Uninstalling setuptools-78.1.0:
      Successfully uninstalled setuptools-78.1.0
Note: you may need to restart the kernel to use updated packages.


<frozen runpy>:128: RuntimeWarning: 'distutils.core' found in sys.modules after import of package 'distutils', but prior to execution of 'distutils.core'; this may result in unpredictable behaviour


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col

In [3]:
spark = SparkSession.builder.appName("GameRecommendation").getOrCreate()


In [4]:

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import pickle


In [5]:
purchased_games = pd.read_csv("./clean_datasets/purchased_games.csv")  # (playerid, library)


In [6]:
purchased_games

,playerid,library
0,76561198060698936,"[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,..."
1,76561198287452552,"[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698..."
2,76561198040436563,"[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130..."
3,76561198042412488,"[300, 240, 220, 320, 360, 4300, 4800, 4000, 61..."
4,76561198119605821,"[47870, 108600, 550, 271590, 331470, 381210, 2..."
...,...,...
102543,76561199063275634,[]
102544,76561198003275888,"[3920, 2600, 6980, 4540, 4550, 7830, 22330, 22..."
102545,76561198944668572,[]
102546,76561198033563710,[]


In [7]:
# purchased_games['library'] = purchased_games['library'].apply(lambda x: x.split(','))


In [8]:
purchased_games

,playerid,library
0,76561198060698936,"[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,..."
1,76561198287452552,"[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698..."
2,76561198040436563,"[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130..."
3,76561198042412488,"[300, 240, 220, 320, 360, 4300, 4800, 4000, 61..."
4,76561198119605821,"[47870, 108600, 550, 271590, 331470, 381210, 2..."
...,...,...
102543,76561199063275634,[]
102544,76561198003275888,"[3920, 2600, 6980, 4540, 4550, 7830, 22330, 22..."
102545,76561198944668572,[]
102546,76561198033563710,[]


In [9]:
all_games = set()
for games in purchased_games["library"]:
    if isinstance(games, str):  # Nếu là chuỗi, tách game
        cleaned_games = [game.strip("[]") for game in games.split(',')]
    elif isinstance(games, list):  # Nếu là danh sách, xử lý trực tiếp
        cleaned_games = [game.strip("[]") for game in games]
    all_games.update(cleaned_games)

game_to_index = {game: idx for idx, game in enumerate(all_games)}
index_to_game = {idx: game for game, idx in game_to_index.items()}

# Tạo ánh xạ player_id → index
player_to_index = {player_id: idx for idx, player_id in enumerate(purchased_games["playerid"])}




In [10]:
all_games

{'',
 ' 755940',
 ' 13530',
 ' 2428310',
 ' 2935700',
 ' 555820',
 ' 2009650',
 ' 1007800',
 ' 1244380',
 ' 2406360',
 ' 521730',
 ' 1748660',
 ' 767420',
 '13600',
 ' 1044340',
 ' 1135030',
 ' 327380',
 ' 613670',
 '510630',
 ' 727640',
 ' 820640',
 ' 490170',
 ' 1701560',
 ' 345650',
 ' 753960',
 ' 898590',
 ' 335450',
 ' 612820',
 ' 1693980',
 ' 528990',
 ' 494980',
 ' 1064070',
 ' 689600',
 ' 1144030',
 ' 94500',
 ' 718490',
 ' 21030',
 ' 2252570',
 ' 983350',
 ' 967390',
 ' 741650',
 ' 1466680',
 ' 426240',
 ' 1373770',
 ' 2914050',
 ' 3242420',
 ' 830470',
 ' 1738990',
 ' 473780',
 ' 960090',
 ' 646010',
 ' 2473710',
 ' 1638440',
 ' 1802340',
 ' 470170',
 ' 521100',
 ' 12120',
 ' 922500',
 ' 591580',
 ' 296850',
 ' 387060',
 ' 1980490',
 ' 476370',
 ' 1587570',
 ' 736110',
 '26900',
 ' 652950',
 '1263370',
 ' 2391710',
 ' 63620',
 ' 1074080',
 ' 757770',
 ' 321360',
 ' 724030',
 ' 438180',
 ' 327930',
 ' 424780',
 ' 979640',
 ' 549090',
 ' 986790',
 ' 1212170',
 ' 671850',
 ' 352

In [11]:
index_to_game

{0: '',
 1: ' 755940',
 2: ' 13530',
 3: ' 2428310',
 4: ' 2935700',
 5: ' 555820',
 6: ' 2009650',
 7: ' 1007800',
 8: ' 1244380',
 9: ' 2406360',
 10: ' 521730',
 11: ' 1748660',
 12: ' 767420',
 13: '13600',
 14: ' 1044340',
 15: ' 1135030',
 16: ' 327380',
 17: ' 613670',
 18: '510630',
 19: ' 727640',
 20: ' 820640',
 21: ' 490170',
 22: ' 1701560',
 23: ' 345650',
 24: ' 753960',
 25: ' 898590',
 26: ' 335450',
 27: ' 612820',
 28: ' 1693980',
 29: ' 528990',
 30: ' 494980',
 31: ' 1064070',
 32: ' 689600',
 33: ' 1144030',
 34: ' 94500',
 35: ' 718490',
 36: ' 21030',
 37: ' 2252570',
 38: ' 983350',
 39: ' 967390',
 40: ' 741650',
 41: ' 1466680',
 42: ' 426240',
 43: ' 1373770',
 44: ' 2914050',
 45: ' 3242420',
 46: ' 830470',
 47: ' 1738990',
 48: ' 473780',
 49: ' 960090',
 50: ' 646010',
 51: ' 2473710',
 52: ' 1638440',
 53: ' 1802340',
 54: ' 470170',
 55: ' 521100',
 56: ' 12120',
 57: ' 922500',
 58: ' 591580',
 59: ' 296850',
 60: ' 387060',
 61: ' 1980490',
 62: ' 47

In [12]:
player_to_index

{76561198060698936: 0,
 76561198287452552: 1,
 76561198040436563: 2,
 76561198042412488: 3,
 76561198119605821: 4,
 76561198049686270: 5,
 76561198155814250: 6,
 76561198083492916: 7,
 76561198150634683: 8,
 76561198836367256: 9,
 76561198131958442: 10,
 76561198106910534: 11,
 76561198122070915: 12,
 76561198740883087: 13,
 76561198016756834: 14,
 76561198072740562: 15,
 76561198144540723: 16,
 76561198096557299: 17,
 76561198884939938: 18,
 76561198220441373: 19,
 76561198046610425: 20,
 76561198801797946: 21,
 76561199000909663: 22,
 76561199004958958: 23,
 76561198120109043: 24,
 76561198856855325: 25,
 76561198146275005: 26,
 76561197989446733: 27,
 76561197997318138: 28,
 76561198015133782: 29,
 76561199158139117: 30,
 76561197962051588: 31,
 76561198191613675: 32,
 76561198080851200: 33,
 76561198230516004: 34,
 76561198282337972: 35,
 76561198391791168: 36,
 76561198213729077: 37,
 76561199248076925: 38,
 76561198234203304: 39,
 76561199225844826: 40,
 76561198087768441: 41,
 7

In [13]:
game_to_index

{'': 0,
 ' 755940': 1,
 ' 13530': 2,
 ' 2428310': 3,
 ' 2935700': 4,
 ' 555820': 5,
 ' 2009650': 6,
 ' 1007800': 7,
 ' 1244380': 8,
 ' 2406360': 9,
 ' 521730': 10,
 ' 1748660': 11,
 ' 767420': 12,
 '13600': 13,
 ' 1044340': 14,
 ' 1135030': 15,
 ' 327380': 16,
 ' 613670': 17,
 '510630': 18,
 ' 727640': 19,
 ' 820640': 20,
 ' 490170': 21,
 ' 1701560': 22,
 ' 345650': 23,
 ' 753960': 24,
 ' 898590': 25,
 ' 335450': 26,
 ' 612820': 27,
 ' 1693980': 28,
 ' 528990': 29,
 ' 494980': 30,
 ' 1064070': 31,
 ' 689600': 32,
 ' 1144030': 33,
 ' 94500': 34,
 ' 718490': 35,
 ' 21030': 36,
 ' 2252570': 37,
 ' 983350': 38,
 ' 967390': 39,
 ' 741650': 40,
 ' 1466680': 41,
 ' 426240': 42,
 ' 1373770': 43,
 ' 2914050': 44,
 ' 3242420': 45,
 ' 830470': 46,
 ' 1738990': 47,
 ' 473780': 48,
 ' 960090': 49,
 ' 646010': 50,
 ' 2473710': 51,
 ' 1638440': 52,
 ' 1802340': 53,
 ' 470170': 54,
 ' 521100': 55,
 ' 12120': 56,
 ' 922500': 57,
 ' 591580': 58,
 ' 296850': 59,
 ' 387060': 60,
 ' 1980490': 61,
 ' 476370

In [16]:
from scipy.sparse import lil_matrix, save_npz, load_npz

num_players = len(purchased_games)
num_games = len(all_games)
interaction_matrix = lil_matrix((num_players, num_games), dtype=np.uint8)

for i, games in enumerate(purchased_games["library"]):
    if isinstance(games, str):  # Nếu games là chuỗi, chuyển nó thành danh sách
        games = games.strip("[]").split(",")  # Tách chuỗi thành danh sách

    for game in games:
        game = str(game).strip()  # Đảm bảo game ID là chuỗi không có khoảng trắng

        if game in game_to_index:  # Kiểm tra game có tồn tại trong game_to_index không
            # if i < 1:     
                # print("Abc")
                # print(game_to_index[game])

            interaction_matrix[i, game_to_index[game]] = 1

# Chuyển ma trận sang CSR (tối ưu tính toán)
interaction_matrix = interaction_matrix.tocsr()

# Lưu ma trận vào file để tái sử dụng mà không cần tính toán lại
save_npz("./data_model/interaction_matrix.npz", interaction_matrix)




In [17]:
from scipy.sparse import lil_matrix, save_npz, load_npz

interaction_matrix = load_npz("./data_model/interaction_matrix.npz")


In [18]:
data = []
for player_id, games in enumerate(interaction_matrix):
    for game_index in games.nonzero()[1]:
        data.append((int(player_id), int(game_index), float(1)))  # Chuyển đổi kiểu dữ liệu

In [19]:
interaction_matrix

<Compressed Sparse Row sparse matrix of dtype 'uint8'
	with 6139343 stored elements and shape (102548, 45093)>

In [20]:
data

[(0, 103, 1.0),
 (0, 136, 1.0),
 (0, 146, 1.0),
 (0, 160, 1.0),
 (0, 181, 1.0),
 (0, 182, 1.0),
 (0, 199, 1.0),
 (0, 205, 1.0),
 (0, 300, 1.0),
 (0, 365, 1.0),
 (0, 504, 1.0),
 (0, 535, 1.0),
 (0, 567, 1.0),
 (0, 569, 1.0),
 (0, 580, 1.0),
 (0, 617, 1.0),
 (0, 619, 1.0),
 (0, 648, 1.0),
 (0, 695, 1.0),
 (0, 703, 1.0),
 (0, 712, 1.0),
 (0, 724, 1.0),
 (0, 725, 1.0),
 (0, 823, 1.0),
 (0, 824, 1.0),
 (0, 826, 1.0),
 (0, 888, 1.0),
 (0, 951, 1.0),
 (0, 1120, 1.0),
 (0, 1173, 1.0),
 (0, 1233, 1.0),
 (0, 1326, 1.0),
 (0, 1331, 1.0),
 (0, 1370, 1.0),
 (0, 1408, 1.0),
 (0, 1425, 1.0),
 (0, 1430, 1.0),
 (0, 1447, 1.0),
 (0, 1503, 1.0),
 (0, 1505, 1.0),
 (0, 1570, 1.0),
 (0, 1587, 1.0),
 (0, 1663, 1.0),
 (0, 1665, 1.0),
 (0, 1675, 1.0),
 (0, 1683, 1.0),
 (0, 1696, 1.0),
 (0, 1732, 1.0),
 (0, 1745, 1.0),
 (0, 1756, 1.0),
 (0, 1764, 1.0),
 (0, 1781, 1.0),
 (0, 1803, 1.0),
 (0, 1884, 1.0),
 (0, 2036, 1.0),
 (0, 2113, 1.0),
 (0, 2194, 1.0),
 (0, 2197, 1.0),
 (0, 2302, 1.0),
 (0, 2324, 1.0),
 (0, 232

In [21]:
# Tạo DataFrame PySpark
columns = ["player_id", "game_id", "rating"]
df = spark.createDataFrame(data, columns)

In [22]:
df = df.withColumn("player_id", col("player_id").cast("int"))
df = df.withColumn("game_id", col("game_id").cast("int"))
df = df.withColumn("rating", col("rating").cast("float"))
df.printSchema()  # Xác nhận lại kiểu dữ liệu


root
 |-- player_id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- rating: float (nullable = true)



In [23]:
df

DataFrame[player_id: int, game_id: int, rating: float]

In [24]:
print(spark)


In [25]:
df.printSchema()


root
 |-- player_id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- rating: float (nullable = true)



In [26]:
from pyspark.sql.functions import col, count

In [27]:
# df.count() # Kiểm tra số dòng trong DataFrame
# print(df.schema)  # Xem kiểu dữ liệu của từng cột


In [28]:
# df.describe().show()  # Kiểm tra thống kê dữ liệu
# df.select([col(c).isNull().sum() for c in df.columns]).show()  # Kiểm tra giá trị null
# df.show(5)  # Xem thử 5 dòng đầu

In [29]:
# Khởi tạo mô hình ALS
from pyspark.ml.recommendation import ALS

als = ALS(
    maxIter=10,         # Số vòng lặp
    regParam=0.1,       # Hệ số điều chuẩn
    userCol="player_id",
    itemCol="game_id",
    ratingCol="rating",
    coldStartStrategy="drop"  # Loại bỏ dữ liệu không đủ thông tin
)

# Huấn luyện mô hình
model = als.fit(df)


Py4JJavaError: An error occurred while calling o59.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (HELLFIRENITRO executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.sca

In [ ]:
interaction_matrix

In [ ]:
interaction_matrix_dense = interaction_matrix.toarray()


In [ ]:
interaction_matrix_dense

In [ ]:

from scipy.sparse.linalg import svds

num_components = 50
U_svd, sigma_svd, Vt_svd = svds(interaction_matrix, k=num_components)

# Chuyển sigma thành ma trận chéo
sigma_svd = np.diag(sigma_svd)

# Lưu kết quả để sử dụng lại
with open("./data_model/svd_model.pkl", "wb") as f:
    pickle.dump((U_svd, sigma_svd, Vt_svd), f)


In [ ]:
# density = interaction_matrix.nnz / (interaction_matrix.shape[0] * interaction_matrix.shape[1])
# print(f"Mật độ dữ liệu không rỗng: {density:.6f}")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_similar_users(player_index, user_embeddings_sparse, top_k=20):
    """
    Lấy danh sách top_k người chơi có độ tương đồng cao nhất với player_index
    """
    similarities = cosine_similarity(user_embeddings_sparse[player_index], user_embeddings_sparse).flatten()
    # print(similarities)
    similar_users = np.argsort(similarities)[::-1][1:top_k + 1]  # Lấy top_k người chơi gần nhất
    return similar_users

In [ ]:
def recommend_games(player_id, U_svd, sigma_svd, Vt_svd, interaction_matrix, 
                    game_to_index, index_to_game, player_to_index, num_recommendations=10):
    
    # Kiểm tra xem player_id có trong player_to_index không
    if player_id not in player_to_index:
        print("❌ Player ID không hợp lệ.")
        return []

    # Chuyển player_id thành index
    player_index = player_to_index[player_id]

    # Tạo sparse embedding để tiết kiệm bộ nhớ
    from scipy.sparse import csr_matrix
    user_embeddings_sparse = csr_matrix(U_svd @ sigma_svd)
    print(user_embeddings_sparse)
    # Lấy danh sách người chơi tương tự
    similar_users = get_similar_users(player_index, user_embeddings_sparse)
    print(similar_users)
    print(similarities[similar_users])
    # Tập hợp các game mà những người chơi tương tự đã mua
    recommended_games = set()
    for similar_user in similar_users:
        recommended_games.update(interaction_matrix[similar_user].nonzero()[1])
    print(recommended_games)
    # Loại bỏ game mà người chơi hiện tại đã mua
    purchased_games = set(interaction_matrix[player_index].nonzero()[1])
    final_recommendations = [index_to_game[idx] for idx in recommended_games if idx not in purchased_games]

    return final_recommendations[:num_recommendations]  # Lấy đúng số lượng cần thiết


In [ ]:
player_id = 76561198060698936  # ID của người chơi cần gợi ý
recommended_games = recommend_games(player_id, U_svd, sigma_svd, Vt_svd, 
                                    interaction_matrix, game_to_index, index_to_game, player_to_index)

print("🎮 Game được đề xuất:", recommended_games)
